# Weaviate Vector Store

#### Creating a Weaviate Client

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [1]:
import weaviate

In [2]:
resource_owner_config = weaviate.AuthClientPassword(
  username = "<username>", 
  password = "<password>", 
)

In [3]:
client = weaviate.Client("https://<cluster-id>.semi.network/", auth_client_secret=resource_owner_config)

#### Load documents, build the GPTVectorStoreIndex

In [ ]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import WeaviateVectorStore
from IPython.display import Markdown, display

In [6]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [ ]:
from llama_index.storage.storage_context import StorageContext


vector_store = WeaviateVectorStore(weaviate_client=client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

# NOTE: you may also choose to define a class_prefix manually.
# class_prefix = "test_prefix"
# vector_store = WeaviateVectorStore(weaviate_client=client, class_prefix=class_prefix)

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

In [13]:
display(Markdown(f"<b>{response}</b>"))

<b>
The author grew up writing short stories, programming on an IBM 1401, and working on microcomputers. He wrote simple games, a program to predict how high his model rockets would fly, and a word processor. He studied philosophy in college, but switched to AI. He reverse-engineered SHRDLU for his undergraduate thesis and wrote a book about Lisp hacking. He visited the Carnegie Institute and realized he could make art that would last. He took art classes at Harvard and applied to RISD and the Accademia di Belli Arti in Florence.</b>